## Save vertices and faces

In [11]:
import trimesh
import torch

mesh_template = trimesh.load_mesh('/home/s94zalek_hpc/shape_matching/data/SMAL_templates/original/template.off',
                                  process=False, validate=False)

faces = torch.tensor(mesh_template.faces, dtype=torch.long)
print(faces.shape)

torch.save(faces,
           '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/data_denoisfm/train/SMAL/faces.pt')

In [12]:
verts = torch.load('/lustre/mlnvme/data/s94zalek_hpc-shape_matching/data_denoisfm/train/SMAL/verts.pt', mmap=True, weights_only=True)
faces = torch.load('/lustre/mlnvme/data/s94zalek_hpc-shape_matching/data_denoisfm/train/SMAL/faces.pt', mmap=True, weights_only=True)

print(verts.dtype, verts.shape)
print(faces.dtype, faces.shape)


In [9]:
scene = trimesh.Scene()

In [ ]:
mesh_i = trimesh.Trimesh(vertices=verts[10000].numpy(), faces=faces.numpy())

scene.geometry.clear()
scene.add_geometry(mesh_i)
scene.show()

## Test caching spectral ops

In [18]:
import importlib
importlib.reload(remesh_util)

In [1]:
import argparse
import logging
import random

import denoisfm.utils.geometry_util as geometry_util
import denoisfm.utils.shape_util as shape_util
import numpy as np
import torch
import denoisfm.utils.remesh_util as remesh_util
from tqdm import tqdm
import os


# parameters for remeshing SMPL or SMAL shapes
config_aug = {
    "isotropic": {
        "simplify_strength_min": 0.2,  # min/max % of ALL faces to keep after simplification
        "simplify_strength_max": 0.8,
    },
    "anisotropic": {
        "probability": 0.0,  # probability of applying anisotropic remeshing
        "fraction_to_simplify_min": 0.2,  # min/max % of faces to SELECT for simplification
        "fraction_to_simplify_max": 0.6,
        "simplify_strength_min": 0.2,  # from the SELECTED faces, min/max % to keep after simplification
        "simplify_strength_max": 0.5,
    },
}

# rewrite args as a class
class Args:
    input_dir = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/data_denoisfm/train/SURREAL'
    n_shapes = 10
    output_dir = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/data_denoisfm/sign_training'

args = Args()



In [2]:
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

##########################################
# Load shapes
##########################################

# load the vertices
shapes_verts = torch.load(f"{args.input_dir}/verts.pt", mmap=True)

# load the faces (same for all shapes)
shapes_faces = torch.load(f"{args.input_dir}/faces.pt", mmap=True)

logging.info("Source shapes loaded")


In [3]:

##########################################
# Generate data
##########################################

# random_idxs = np.random.choice(len(shapes_verts), args.n_shapes, replace=False)

random_idxs = range(10, 110, 10)

# remove the output directory if it exists
if os.path.exists(args.output_dir):
    os.system(f"rm -r {args.output_dir}")

dir_off = f"{args.output_dir}/off"
dir_spectral = f"{args.output_dir}/diffusion"
os.makedirs(dir_off, exist_ok=True)
os.makedirs(dir_spectral, exist_ok=True)

for i in tqdm(range(args.n_shapes), desc="Generating data"):
    
    # remesh the shape
    verts, faces, corr = remesh_util.augmentation_pipeline(
        verts_orig=shapes_verts[random_idxs[i]],
        faces_orig=shapes_faces,
        config=config_aug,
    )
    # rotation and scaling
    verts_aug = geometry_util.data_augmentation(
        verts.unsqueeze(0),
        rot_x=0,
        rot_y=90,  # random rotation around y-axis
        rot_z=0,
        std=0,  # no random noise
        scale_min=0.9,  # random scaling
        scale_max=1.1,
    )[0]

    # save the mesh
    shape_util.write_off(
        f"{dir_off}/{i:04}.off", verts_aug.cpu().numpy(), faces.cpu().numpy()
    )
    
    # read the mesh again
    # verts_aug, faces = shape_util.read_shape(f"{dir_off}/{i:04}.off")
            
    # verts_aug = torch.tensor(verts_aug, dtype=torch.float32)
    # faces = torch.tensor(faces, dtype=torch.long)
    
    # calculate and cache the laplacian
    geometry_util.get_operators(verts_aug, faces, k=128, cache_dir=dir_spectral)


In [4]:
def load_cached_shapes(input_dir, k_eig):
    off_dir = f"{input_dir}/off"
    spectral_dir = f"{input_dir}/diffusion"

    # get all meshes in the folder
    off_files = sorted([f for f in os.listdir(off_dir) if f.endswith(".off")])

    shapes_list = []
    for file in tqdm(off_files, desc="Loading shapes and spectral operators"):
        # load the vertices and faces
        verts, faces = shape_util.read_shape(os.path.join(off_dir, file))
        verts = torch.tensor(verts, dtype=torch.float32)
        faces = torch.tensor(faces, dtype=torch.long)

        # load the spectral operators cached in spectral_dir
        _, mass, L, evals, evecs, gradX, gradY = geometry_util.get_operators(
            verts, faces, k=k_eig, cache_dir=spectral_dir
        )
        shapes_list.append(
            {
                "verts": verts,
                "faces": faces,
                "evecs": evecs,
                "mass": mass,
                "L": L,
                "evals": evals,
                "gradX": gradX,
                "gradY": gradY,
            }
        )
    return shapes_list


In [5]:
load_cached_shapes(args.output_dir, 128)

In [7]:
import trimesh

mesh = trimesh.load('/home/s94zalek_hpc/DenoisingFunctionalMaps/data/template_human/template.off')

verts = torch.tensor(mesh.vertices)
faces = torch.tensor(mesh.faces)

_, mass, L, evals, evecs, gradX, gradY = geometry_util.get_operators(
            verts, faces, k=128
        )
# 

In [8]:
evecs

In [10]:
from denoisfm.sign_correction import area_normalize

evecs_norm = area_normalize(evecs, mass)

In [13]:
(evecs - evecs_norm).abs().sum()

In [15]:
torch.nn.functional.normalize(evecs, p=2, dim=1)